# Run PncA WandB Sweep

In [1]:
from IPython.display import display
import os

if "SSH_CONNECTION" in os.environ:
    display("Running via SSH")
else:
    display("Running locally")
    
import sys
import os

path = os.path.join('..', '/Users/dylandissanayake/Desktop/DPhil/Comp Disc/Repositories/TB-PNCA-GNN') if "SSH_CONNECTION" not in os.environ else os.path.join('..', '/mnt/alphafold-volume-1/dylan2/repos/tb-pnca-gnn')
if path not in sys.path:
    sys.path.append(os.path.abspath(path))

import datetime
import random

import numpy as np
import pandas as pd
import copy
import pickle

import torch
from torch_geometric.data import Data

import wandb

import sbmlcore
import sbmlsim
import gumpy

import warnings
warnings.filterwarnings('ignore')

from src import run_model, protein_graph, gcn_model, evaluation

%load_ext autoreload
%autoreload 2

%aimport src

torch.cuda.is_available()

'Running via SSH'

/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
/home/ubuntu/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


True

In [2]:
full_dataset = torch.load('full_orig_dataset.pth')

### Set Up Params and Sweep Config

In [5]:
seed = 42
np.random.seed(seed)
random.seed(seed)

# logging params (only used for wandb metrics)
n_samples = 10000
cutoff_distance = 6.3  

# gcn params
ews = 'none'
num_node_features = 12
batch_size = 256
hidden_channels = 64
learning_rate = 0.001
wd = 5e-5
epochs = 500

wt_seq = 'MRALIIVDVQNDFCEGGSLAVTGGAALARAISDYLAEAADYHHVVATKDFHIDPGDHFSGTPDYSSSWPPHCVSGTPGADFHPSLDTSAIEAVFYKGAYTGAYSGFEGVDENGTPLLNWLRQRGVDEVDVVGIATDHCVRQTAEDAVRNGLATRVLVDLTAGVSADTTVAALEEMRTASVELVCS'

In [4]:
# First sweep:

sweep_config = {
    'method': 'random'
    }

metric = {
    'name': 'Test Accuracy',
    'goal': 'maximize'   
    }

sweep_config['metric'] = metric

parameters_dict = {
    'hidden_channels': {
        'values': [32, 64, 128, 256]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-2
        },
    'dropout': {
          'values': [0.2, 0.4, 0.5, 0.6, 0.8]
        },
    'cutoff_distance': {
        'distribution': 'uniform',
        'min': 3,
        'max': 10 
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-8,
        'max': 1e-1
      },
    }

sweep_config['parameters'] = parameters_dict

# Second / third sweep:

parameters_dict.update({
    'hidden_channels': {
        'values': [64, 128, 192, 256, 320, 384]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 5e-7, 
        'max': 1e-2
        },
    'dropout': {
          'values': [0.4, 0.5, 0.6, 0.8]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-2 
        },
    })

# Fourth sweep:

parameters_dict.update({
    'hidden_channels': {
        'values': [128, 192, 256, 320]
        },
    'weight_decay': {
        'distribution': 'log_uniform_values',
        'min': 1e-5,
        'max': 1e-3
        },
    'dropout': {
          'values': [0.4, 0.5, 0.6, 0.8]
        },
    'learning_rate': {
        'distribution': 'log_uniform_values',
        'min': 5e-5,
        'max': 1e-3
        },
    'cutoff_distance': {
        'distribution': 'uniform',
        'min': 3,
        'max': 9
        },
    })



In [5]:
import pprint
pprint.pprint(sweep_config)

{'method': 'random',
 'metric': {'goal': 'maximize', 'name': 'Test Accuracy'},
 'parameters': {'cutoff_distance': {'distribution': 'uniform',
                                    'max': 9,
                                    'min': 3},
                'dropout': {'values': [0.4, 0.5, 0.6, 0.8]},
                'hidden_channels': {'values': [64, 128, 192, 256, 320]},
                'learning_rate': {'distribution': 'log_uniform_values',
                                  'max': 0.001,
                                  'min': 5e-05},
                'weight_decay': {'distribution': 'log_uniform_values',
                                 'max': 0.001,
                                 'min': 1e-05}}}


### Define Training Loop

In [8]:
project = "pnca-sweep-1"

In [9]:
sweep_id = wandb.sweep(sweep_config, project=project)
# sweep_id = '18ili8gf'
# sweep_id = 'z95xpbwd'
# sweep_id = 'tr8mwg42'

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Create sweep with ID: tr8mwg42
Sweep URL: https://wandb.ai/dylan-home/pnca-sweep-1/sweeps/tr8mwg42


In [13]:
def sweep_run():

    with wandb.init() as run:
        config = run.config
        
        model = run_model.pnca_simpleGCN(
            # sequences=sequences_dict,
            sequences=None,
            self_loops = False,
            cutoff_distance = config.cutoff_distance,
            edge_weight_func = '1-(dist/cutoff)',
            # edge_weight_func = 'none',
            batch_size = batch_size,
            num_node_features = num_node_features,
            hidden_channels = config.hidden_channels,
            learning_rate = config.learning_rate,
            wd = config.weight_decay,
            dropout = config.dropout,
            epochs = epochs,
            dataset = full_dataset,
            normalise_ews=True,
            wandb_params={
                'use_wandb': False,
                'sweep': True
            }
        )

        torch.save(model, f'saved_models/carter_ds_aug/{project}/{datetime.datetime.now().strftime("%Y_%m_%d_%H%M%S")}')

In [14]:
wandb.agent(sweep_id, sweep_run, project = project, count=200)

wandb: Agent Starting Run: vm0xw36w with config:
wandb: 	cutoff_distance: 7.1589747289868555
wandb: 	dropout: 0.4
wandb: 	hidden_channels: 64
wandb: 	learning_rate: 9.031200079445804e-05
wandb: 	weight_decay: 0.0027336118535111185
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "
wandb: Currently logged in as: dylan-home. Use `wandb login --relogin` to force relogin


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.4949, Test Acc: 0.4817, Train Loss: 0.6922, Test Loss: 0.6915
Epoch: 020, Train Acc: 0.5101, Test Acc: 0.5023, Train Loss: 0.6907, Test Loss: 0.6902
Epoch: 030, Train Acc: 0.4841, Test Acc: 0.4703, Train Loss: 0.6905, Test Loss: 0.6890
Epoch: 040, Train Acc: 0.4650, Test Acc: 0.4653, Train Loss: 0.6898, Test Loss: 0.6878
Epoch: 050, Train Acc: 0.6244, Test Acc: 0.6203, Train Loss: 0.6899, Test Loss: 0.6868
Epoch: 060, Train Acc: 0.5256, Test Acc: 0.5107, Train Loss: 0.6876, Test Loss: 0.6878
Epoch: 070, Train Acc: 0.5253, Test Acc: 0.5107, Train Loss: 0.6871, Test Loss: 0.6871
Epoch: 080, Train Acc: 0.5236, Test Acc: 0.5820, Train Loss: 0.6906, Test Loss: 0.6848
Epoch: 090, Train Acc: 0.5184, Test Acc: 0.5093, Train Loss: 0.6828, Test Loss: 0.6824
Epoch: 100, Train Acc: 0.4753, Test Acc: 0.4787, Train Loss: 0.6796, Test Loss: 0.6767
Epoch: 110, Train Acc: 0.4797, Test Acc: 0.4837, Train Loss: 0.6769

Test Accuracy,▂▄▂▁▂▂▄▂▂▁▆▂▂▇▆▁▇▆▇▇▆▇▇▇▂▇▆▇█▇▇█████████
Test F1,▇▃▇▆▇▆▁▇▇▆▇▆▆▇▅▆▇▅▆▇▆▆▆▆▆█▅▇██▇██▇███▇█▇
Test Loss,████████▇▇▇▇▇▆▆▆▅▅▅▄▄▄▄▃▄▄▄▂▃▃▂▂▂▂▃▂▂▁▁▁
Test Sensitivity,█▂█▆██▁██▆▅█▆▅▃▇▅▄▄▅▄▄▄▄▇▇▃▅▆▇▄▆▅▅▇▆▆▅▆▄
Test Specificity,▁█▁▂▁▁█▁▁▂▆▁▂▆▇▁▇▇▇▇▇▇▇▇▂▅█▇▆▅▇▆▆▇▅▆▆▇▇▇
Train Accuracy,▂▄▂▁▂▂▂▂▂▁▆▂▂▆▅▂▆▅▆▇▆▆▇▇▃▇▅▇██▇██▇█████▇
Train F1,▇▃▇▆▆▆▁▇▆▆▇▆▆▇▅▆▇▅▇▇▆▆▇▇▆█▅▇██▇██▇███▇█▇
Train Loss,█████████▇▇▇▇▇▇▆▆▆▅▅▅▅▄▄▅▄▄▃▃▃▂▂▂▂▃▂▂▁▁▂
Train Sensitivity,█▂█▆██▁██▆▅█▆▆▄▇▅▄▅▅▄▄▅▅▇▇▃▅▆▇▅▆▆▅▇▇▇▅▆▅
Train Specificity,▁█▁▂▁▁█▁▁▂▆▁▂▆█▁▇█▇▇█▇▇▇▂▅█▇▇▆▇▇▇▇▆▆▇▇▇█
Test Accuracy,0.74667


wandb: Agent Starting Run: 16bjl0ar with config:
wandb: 	cutoff_distance: 3.6269286906904816
wandb: 	dropout: 0.4
wandb: 	hidden_channels: 64
wandb: 	learning_rate: 0.0059142339243303445
wandb: 	weight_decay: 0.002533177463390691
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6922, Test Loss: 0.6938
Epoch: 020, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6917, Test Loss: 0.6935
Epoch: 030, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6919, Test Loss: 0.6937
Epoch: 040, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6919, Test Loss: 0.6938
Epoch: 050, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6918, Test Loss: 0.6935
20 epochs passed without 0 test loss improvement. 
Early stopping triggered.


...

wandb: Agent Starting Run: s0uwutjt with config:
wandb: 	cutoff_distance: 4.907781220367518
wandb: 	dropout: 0.2
wandb: 	hidden_channels: 128
wandb: 	learning_rate: 0.005154436174204434
wandb: 	weight_decay: 1.681425870548816e-05
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6918, Test Loss: 0.6933
Epoch: 020, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6918, Test Loss: 0.6933
Epoch: 030, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6920, Test Loss: 0.6930
Epoch: 040, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6923, Test Loss: 0.6929
Epoch: 050, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6920, Test Loss: 0.6930
Epoch: 060, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6916, Test Loss: 0.6935
20 epochs passed without 0 test loss improvement. 
Early stopping triggered.


Test Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test F1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Loss,▂▃▅▂▆▂▄▃▄▂▂▄▃▃▅█▇▃▅▂▇▆▁▂▃▂▁▄▁▆▄▃▃▂▅▁▂▃▄▅
Test Sensitivity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Specificity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Accuracy,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train F1,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Loss,▄▅▅▃▃▃▄▅▄▃▃▂▄▄▅▅▇▃▅▄▁▅▆▄▅▅█▄▇▄▄▃▃▅▅▅▅▄▄▂
Train Sensitivity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Train Specificity,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Test Accuracy,0.511


wandb: Agent Starting Run: sv5fpykc with config:
wandb: 	cutoff_distance: 6.884253817876914
wandb: 	dropout: 0.6
wandb: 	hidden_channels: 192
wandb: 	learning_rate: 0.0003282163786724674
wandb: 	weight_decay: 1.215187019975012e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
/usr/lib/python3/dist-packages/requests/__init__.py:87: RequestsDependencyWarning: urllib3 (2.2.3) or chardet (4.0.0) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({}) doesn't match a supported "


Using CUDA
Early stopping enabled. Patience: 20. Min Delta: 0.
Epoch: 010, Train Acc: 0.5040, Test Acc: 0.5520, Train Loss: 0.6921, Test Loss: 0.6894
Epoch: 020, Train Acc: 0.5264, Test Acc: 0.5110, Train Loss: 0.6887, Test Loss: 0.6894
Epoch: 030, Train Acc: 0.5134, Test Acc: 0.4890, Train Loss: 0.6860, Test Loss: 0.6847
Epoch: 040, Train Acc: 0.5263, Test Acc: 0.5110, Train Loss: 0.6840, Test Loss: 0.6844
Epoch: 050, Train Acc: 0.5044, Test Acc: 0.5373, Train Loss: 0.6934, Test Loss: 0.6832
Epoch: 060, Train Acc: 0.5179, Test Acc: 0.5003, Train Loss: 0.6701, Test Loss: 0.6695
Epoch: 070, Train Acc: 0.5560, Test Acc: 0.6197, Train Loss: 0.6770, Test Loss: 0.6682
Epoch: 080, Train Acc: 0.5224, Test Acc: 0.5067, Train Loss: 0.6602, Test Loss: 0.6637
Epoch: 090, Train Acc: 0.5341, Test Acc: 0.5040, Train Loss: 0.6248, Test Loss: 0.6237
Epoch: 100, Train Acc: 0.7533, Test Acc: 0.7253, Train Loss: 0.6045, Test Loss: 0.6044
Epoch: 110, Train Acc: 0.7646, Test Acc: 0.7383, Train Loss: 0.5831